In [1]:
import yaml
from util.thchs30_dataset import load_dataset,create_dataloader
from util.functions import log_parser,batch_iterator, collapse_phn
from model.las_model import Listener,Speller
import numpy as np
from torch.autograd import Variable
import torch
import time
import pdb
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import python_speech_features as features
import os
import pickle
from util.functions import LetterErrorRate
from six.moves import cPickle
from preprocess_all_datasets import preprocess_dataset

In [2]:
import librosa
import IPython.display as ipd
def play_audio(path):
    """Plays the audio of the give path
       within Jupyter Notebook
    """
    samples, sample_rate = librosa.load(path, mono = True, sr = None)
    return ipd.Audio(samples, rate = sample_rate)

### 读音频文件

In [3]:
wav_file_path='/data_sdd/audio_data/data_thchs30/data/D8_999.wav'
phn_file_path='/data_sdd/audio_data/data_thchs30/data/D8_999.wav.trn'

In [4]:
play_audio(wav_file_path)

In [5]:
##load dict file //dict_zh_words.py
dict_path = '/share_sdb/hxx/ASR_ZH/LAS-Chinese-pytorch/data/all_dict.pkl'
with open(dict_path, 'rb') as f:
    all_dict = pickle.load(f)
index2word = {k: v for k, v in enumerate(all_dict)}

### TESTING

In [8]:
# Load example config file for experiment
config_path = 'config/las_all_datasets_config.yaml'
conf = yaml.load(open(config_path,'r'))
listener_model_path = conf['meta_variable']['checkpoint_dir']+conf['meta_variable']['experiment_name']+'.listener'
speller_model_path = conf['meta_variable']['checkpoint_dir']+conf['meta_variable']['experiment_name']+'.speller'
listener = torch.load(listener_model_path)
speller = torch.load(speller_model_path)

X_val = wav_file_path; y_val=[]
X_test = []; y_test = []
X_test.append(X_val); y_test.append(y_val)
test_set = create_dataloader(X_test, y_test, **conf['model_parameter'], **conf['testing_parameter'], shuffle=False)

In [9]:
for batch_index,(batch_data,batch_label) in enumerate(test_set):
    #pdb.set_trace()
    feature = listener(Variable(batch_data.float()).cuda())
    pred_seq, _ = speller(feature)
    
    words_num_result = np.argmax(np.array([p.data.cpu().numpy() for p in pred_seq]).transpose(1,0,2),axis=-1).tolist()
    words_result = []
    i=0
    while i<len(words_num_result[0]):
        if not (words_num_result[0][i]-2) in index2word :
            break
        words_result.append(index2word[(words_num_result[0][i]-2)] if (words_num_result[0][i]-2) in index2word  else '-')
        i=i+1

In [10]:
','.join(words_result)

'国,务,员,监,国,务,院,秘,书,长,罗,干,民,政,部,长,多,机,材,让,也,一,同,前,往,延,安,看,往,人,民,群,众'

In [ ]:
thchs30_dirs_test = '/data_sdd/audio_data/data_thchs30/test/'
X_test_thchs30, y_test_thchs30 = preprocess_dataset(thchs30_dirs_test)
test_set_thchs30 = create_dataloader(X_test_thchs30, y_test_thchs30, **conf['model_parameter'], **conf['testing_parameter'], shuffle=False)
test_ler = []
for batch_index,(batch_data,batch_label) in enumerate(test_set_thchs30):
    feature = listener(Variable(batch_data.float()).cuda())
    pred_seq, _ = speller(feature)
    
    batch_ler = LetterErrorRate(np.argmax(np.array([p.data.cpu().numpy() for p in pred_seq]).transpose(1,0,2),axis=-1),
                                np.argmax(batch_label.numpy(),axis=-1),data='thchs30')
    test_ler.extend(batch_ler)

print('Testing Phoneme Error Rate',sum(test_ler)/len(test_ler))